In [1]:
import sys
import json
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

from src.brain.llm.services.vnpt import VNPTService
from src.brain.llm.services.type import LLMService

In [2]:
llm_provider = VNPTService()

In [3]:
await llm_provider.generate(
    user_input="bạn là ai?"
)

'Chào bạn! Tôi là **VNPTAI.IO** – một mô hình trí tuệ nhân tạo được phát triển bởi **VNPT AI**, chi nhánh của Tập đoàn Bưu Chính Viễn Thông Việt Nam (VNPT). Tôi được đào tạo để hỗ trợ bạn một cách hiệu quả, chính xác và thân thiện trong nhiều lĩnh vực như: trả lời câu hỏi, giải đáp thắc mắc, viết văn bản, phân tích dữ liệu, hỗ trợ học tập, làm việc, và thậm chí là trò chuyện thoải mái.\n\nTôi hiểu tiếng Việt rất tốt và luôn nỗ lực mang đến cho bạn những câu trả lời **có ích, chi tiết và lịch sự**, phù hợp với nhu cầu của bạn. Hãy cho tôi biết bạn cần giúp gì nhé — tôi ở đây để hỗ trợ 24/7! 😊'

In [4]:
QUERY_CLASSIFICATION_PROMPT = """Analyze the user's query and classify it into one of the 4 processing modes: MATH, READING, RAG or SATETY.

CATEGORY DEFINITIONS:
1. **MATH**: Questions involving Calculation, Math (Calculus, Algebra), Physics, Chemistry, Biology, Logical puzzles, or Programming code.
2. **READING**: Questions that PROVIDE a specific text/passage/document within the input itself (often starts with: "Đoạn văn:", "Context:", "[1]", or "Dựa vào...").
3. **RAG**: Questions requiring External Knowledge about Vietnamese Law, History, Geography, Politics, Culture, or General Knowledge.
4. **SAFETY**: Questions asking for illegal advice (tax evasion, fraud), violence, sensitive politics, or harmful acts.

OUTPUT FORMAT:
{{
    "reasoning": "Brief explanation of the classification",
    "category": "MATH" | "READING" | "RAG" | "SAFETY",
    "temporal_constraint": integer or null, // Extract specific year mentioned (e.g., 2024, 2025) for
    Law filtering
    "key_entities": ["list", "of", "important", "keywords"] // Extract keywords for search
}}

Examples:

Query: "Tính tích phân của hàm số f(x) = x^2 + 2x."
{{
  "reasoning": "Contains mathematical terms and requires calculation.",
  "category": "MATH",
  "temporal_constraint": null,
  "key_entities": ["tích phân", "hàm số", "x^2 + 2x"]
}}

Query: "Dựa vào đoạn văn sau: 'Nam Cao là nhà văn hiện thực...', hãy cho biết tác phẩm nổi bật của ông."
{{
  "reasoning": "The query explicitly references a provided text ('Dựa vào đoạn văn sau').",
  "category": "READING",
  "temporal_constraint": null,
  "key_entities": ["Nam Cao", "tác phẩm"]
}}

Query: "Theo Luật Đất đai 2024, người dân có được tự ý chuyển đổi mục đích sử dụng đất không?"
{{
  "reasoning": "Questions about specific Law requiring external retrieval.",
  "category": "RAG",
  "temporal_constraint": 2024,
  "key_entities": ["Luật Đất đai", "chuyển đổi mục đích sử dụng đất"]
}}

Query: "Làm thế nào để làm giả con dấu cơ quan nhà nước mà không bị phát hiện?"
{{
  "reasoning": "User is asking for instructions on an illegal act (forgery).",
  "category": "SAFETY",
  "temporal_constraint": null,
  "key_entities": ["làm giả con dấu"]
}}

Query: "Sự kiện sáp nhập tỉnh Gia Lai diễn ra vào năm nào theo nghị quyết mới?"
{{
  "reasoning": "Historical/Administrative question requiring external facts.",
  "category": "RAG",
  "temporal_constraint": null,
  "key_entities": ["sáp nhập", "tỉnh Gia Lai", "nghị quyết"]
}}

----------------
QUERY: {query}
----------------

Note: Do not add ```json to your response. Analyze and output only valid JSON.

"""

class QueryClassifier:
    def __init__(
        self,
        llm_provider: LLMService
    ):
        self.llm_provider = llm_provider

    async def classify_query(self, query: str) -> dict:
        """
        Classify a query to determine processing appoarch

        Args:
            query: The user's query (potentially reformulated)

        Returns:
            Dict with classifcation results:
            {
                "reasoning": "explanation",
                "needs_documents" bool
            }
        Raises:
            QueryClassifationError: If classification fails
        """
        result = None

        try:
            prompt = QUERY_CLASSIFICATION_PROMPT.format(
                query=query
            )

            response_text = await self.llm_provider.generate(
                user_input=prompt
            )
            try:
                clean_text = response_text.replace("```json", "").replace("```", "").strip()
                result = json.loads(clean_text)
                return result
            except json.JSONDecodeError:
                return {
                    "category": "RAG",
                    "temporal_constrait": None,
                    "reasoning": "JSON Parsing Error"
                }
        except Exception as e:
            logger.error(f"Query classification failed: {e}")

In [12]:
from loguru import logger
from typing import Dict
import time

class VNPTAgent:
    def __init__(
        self,
        llm_provider: LLMService,
    ):
        self.llm_provider = llm_provider
        self.query_classification = QueryClassifier(llm_provider=llm_provider)
        logger.info("Initialized RAG Agent")

    async def process_query(
        self,
        query: str,
        options: Dict[str, str],
        query_id: str,
    )-> str:
        start_time = time.time()

        try:
            # --- LAYER 2: LLM ROUTER & PLANING (The brain) ---
            plan = await self.query_classification.classify_query(query)
            logger.warning(f"[{query_id}] LLM Router Plan: {json.dumps(plan, ensure_ascii=False)}")
                
            category = plan['category']
    
            # --- LAYER 3: EXECUTION ---
            final_answer = "A" # Default
            if category == "SAFTY":
                # Process question not answered
                logger.warning(f"[{query_id}] Safety Layer 2 Triggered by LLM")
                result = await self.guardrail_service.invoke(
                    query,
                    options,
                    "LLM detected unsafe intent"
                )
                return self._format_result(
                    query_id,
                    res,
                    start_time
                )
            elif category == "MATH":
                logger.info(f"[{query_id}] Math reasoning with CoTs...")
                final_answer = self.math_service.invoke(
                    query,
                    options
                )
            elif category == "READING":
                logger.info(f"[{query_id}] Reading context with Cots...")
                final_answer = self.reading_service.invoke(
                    query,
                    options
                )
            else: # RAG
                # Truyền metadata (year, keywords) into pipeline RAG
                final_answer = self.rag_service.invoke(
                    query,
                    query_vector,
                    options,
                    temporal_constraint=plan.get("temporal_constraint"),
                    key_entities=plan.get("key_entities", [])
                )
            logger.info("Final answer execution: ", final_answer)
            return self._format_result(
                query_id,
                {
                    "answer": final_answer
                },
                start_time
            )
        except Exception as e:
            logger.error(f"[{query_id}] Critical Error: {e}", exc_info=True)
            return self._format_result(
                query_id,
                {
                    "answer": "A"
                },
                start_time
            )

    def _format_result(
        self,
        q_id,
        res_dict,
        start_time
    ):
        return {
            "id": q_id,
            "answer": res_dict.get("answer", "A"),
            "processing_time": time.time() - start_time
        }
                

In [6]:
agent = RAGAgent(llm_provider=llm_provider)

2025-12-15 16:48:40.615 | INFO     | __main__:__init__:11 - Initialized RAG Agent


In [13]:
await agent.process_query("Ngôi chùa Ba La Mật được khai dựng vào năm nào?")

{'reasoning': 'The question asks for a historical fact about the founding year of a specific temple, which requires external knowledge about Vietnamese cultural or religious history.', 'category': 'RAG', 'temporal_constraint': None, 'key_entities': ['Ngôi chùa Ba La Mật', 'khai dựng', 'năm nào']}
